On this notebook I'll test the random forest algorithm (the one with better performance) with outside data - other songs from playlists - meaning neither data from the traning nor the test set.

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import json
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from scipy import stats
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind

#### Connecting to Spotify

In [2]:
username = 'britopereiraa'
client_id ='17a3ed14ab434b33b28f19f8bea3337d'
client_secret = '7f848f9d2a024b15be5fa776b4dc7a05'
redirect_uri = 'http://localhost:7777/callback'
scope = 'user-read-recently-played'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

In [3]:
print(token)

BQA5hNVXOcNjQz3c47G_d6dtWwDSKBPzXPszYiLbw3UvpfqAYLPQZx0_LiZB--mgk2oIwMKISdmwYiIbRdZRYRfNRshRNhzbZiyq1lutaESUBfoJqrFsW4Z_q8KNS6b654GO08s8sPYgKh8NiFnN9I5RLe8


In [4]:
credentials = json.load(open('authorization.json'))
client_id = credentials['client_id']
client_secret = credentials['client_secret']

#### Test Random Forest

In order to test my best classifier (Random Forest):

In [18]:
#using the best algorithm (have to train again...)

data = pd.read_csv('../data/my_data.csv')

features = ['danceability', 'energy', 'key','loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_ms']

X = data[features]
y = data['Like']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

metrics.accuracy_score(y_test, y_pred)

0.8768656716417911

###  Vodafone FM 

https://open.spotify.com/playlist/5QZyi299jBblzaTFW1nP4h?si=PawXR_5mSL-ZfezTCUX2Hg

In [19]:
#I'm going to test with a playlist from Vodafone FM 

playlist_index = 0

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri

#just to confirm: spotify:playlist:3M2gjOU19KrxZOCYtVYUCY

'spotify:user:Test_1:playlist:3M2gjOU19KrxZOCYtVYUCY'

In [20]:
def master_function(uri):
    uri = playlist_uri    
    username = uri.split(':')[2]
    playlist_id = uri.split(':')[4]
    results = {'items':[]}

    for n in range(0,3000,100):
        new = sp.user_playlist_tracks(username, playlist_id,  offset = n)
        results['items'] += new['items']

        playlist_tracks_data = results
        playlist_tracks_id = []
        playlist_tracks_titles = []
        playlist_tracks_artists = []

        for track in playlist_tracks_data['items']:
            playlist_tracks_id.append(track['track']['id'])
            playlist_tracks_titles.append(track['track']['name'])

        #adds a list of all artists involved in the song to the list of artists for the playlist
            for artist in track['track']['artists']:
                artist_list = []
                artist_list.append(artist['name'])
            playlist_tracks_artists.append(artist_list[0])
    
    df = pd.DataFrame([])
    for i in range(0, len(playlist_tracks_id)):
        features = sp.audio_features(playlist_tracks_id[i])
        features_df = pd.DataFrame(features)
        df = df.append(features_df)

    df['title'] = playlist_tracks_titles
    #features_df['first_artist'] = playlist_tracks_first_artists
    df['main_artist'] = playlist_tracks_artists
    #features_df = features_df.set_index('id')
    df = df[['id', 'title', 'main_artist',
                               'danceability', 'energy', 'key', 'loudness',
                               'mode', 'acousticness', 'instrumentalness',
                               'liveness', 'valence', 'tempo',
                               'duration_ms', 'time_signature']]

    return df

In [21]:
test_df = master_function(playlist_uri)

In [22]:
test_df.shape

(292, 15)

In [23]:
#passing with the test playlist 

pred = clf.predict_proba(test_df[features])[:,1]

test_df['prediction'] = pred
test_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.707,2,-7.626,1,0.6250,0.03550,0.116,0.502,143.802,216004,4,0.98
0,07LpPO7qBMtJGZ3fz5Khxw,Hi-Five,Angel Olsen,0.514,0.574,9,-7.224,1,0.0443,0.00826,0.142,0.369,125.389,177520,4,0.74
0,6hlJIYn5IBNqcSGTS6X5Xq,My Girls,Animal Collective,0.508,0.953,4,-6.031,0,0.2560,0.04450,0.929,0.555,119.973,340787,4,0.37
0,6bZaumwiWjUSLSLqUyxetj,The Suburbs,Arcade Fire,0.618,0.676,6,-6.127,0,0.0577,0.01970,0.343,0.537,117.783,315200,4,0.68
0,4VAkPXhI7xQ6FHlUuY3RzB,Wake Up,Arcade Fire,0.254,0.787,0,-4.813,1,0.1110,0.04490,0.105,0.301,139.389,335333,4,0.55


In [24]:
print("How similar is it to my taste?", round(test_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 47.154 %


In [25]:
liked_test_songs = test_df[test_df['prediction'] >0.9]
liked_test_songs

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,0HV1cGoX5ASBlPA1pqH8fG,Every Other Freckle,alt-J,0.528,0.7070,2,-7.626,1,0.6250,0.035500,0.1160,0.5020,143.802,216004,4,0.98
0,70WyPf8WtXvG3CgeTZZWz6,Nantes,Beirut,0.541,0.5330,6,-9.575,1,0.7860,0.010200,0.1380,0.6420,131.147,230200,4,0.96
0,3Ay3OuDTieyXTNibBO0mBy,Red Diamond,Best Youth,0.569,0.3600,9,-10.418,0,0.6110,0.001130,0.1130,0.1730,118.983,256134,4,0.99
0,0kHoisnf1ddnV6MrFLWka9,Hang Out,Best Youth,0.514,0.4520,6,-10.420,0,0.1940,0.455000,0.1770,0.2000,139.885,264000,4,0.95
0,7fXyuQqEBwpKtCa8PomO7B,Skinny Love,Bon Iver,0.590,0.2600,4,-14.065,0,0.8500,0.000005,0.1170,0.1120,77.118,238520,4,0.99
0,7oSnuFmMKbC863bqZ0atce,Première,Bruno Pernadas,0.560,0.3740,7,-15.621,1,0.2490,0.060500,0.3680,0.1560,92.031,490776,4,0.94
0,4ccIlWJ2R0FAI8bDyFdMTu,Our Love,Caribou,0.729,0.4950,10,-10.747,0,0.1310,0.886000,0.1020,0.4830,125.002,334173,4,0.96
0,0iHwnccPp214om2eGT5gzm,1998,Chet Faker,0.785,0.3820,1,-13.033,1,0.3480,0.344000,0.0882,0.5080,99.983,365382,4,0.97
0,1MJ5f5EYBC92ADD6xcz7nb,Chamber Of Reflection,Mac DeMarco,0.537,0.5580,11,-8.678,1,0.2630,0.910000,0.1020,0.5050,131.037,231724,4,0.95
0,1QNUdBrQyfboTVqaf4R3hL,A pele que há em mim (Quando o dia entardeceu)...,JP Simões,0.637,0.1580,9,-13.508,1,0.9240,0.003620,0.1110,0.0919,104.502,234280,3,0.99


### Test Felipe's taste 

https://open.spotify.com/playlist/12Wbv8sIx84T5uh6iOoJ7V?si=aevZa-w7TqS-MFGoDmn0eQ

In [26]:
#I'm going to test with a playlist from Felipe: 'Uma'

playlist_index = 1

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri
#spotify:playlist:12Wbv8sIx84T5uh6iOoJ7V

'spotify:user:Test_2:playlist:12Wbv8sIx84T5uh6iOoJ7V'

In [27]:
felipe_df = master_function(playlist_uri)

In [28]:
felipe_df = felipe_df.drop(columns=['time_signature'])

In [29]:
#passing with the Felipe's playlist 
pred = clf.predict_proba(felipe_df[features])[:,1]

felipe_df['prediction'] = pred
felipe_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,prediction
0,0A0eOcimSNRs2EQQlH7FFJ,Três Dias,Marcelo Camelo,0.291,0.319,7,-10.465,1,0.7910,0.46800,0.0692,0.038,103.793,323280,0.97
0,0MtVmhAx6CxNuxFIUc6Mj9,Elephant Gun,Beirut,0.171,0.626,8,-8.677,1,0.8730,0.02520,0.0681,0.457,180.098,348893,0.70
0,6aUAF8JOd8zEl41B6I18xL,Fake Empire,The National,0.324,0.776,0,-6.784,1,0.1510,0.91700,0.0728,0.317,101.964,205040,0.69
0,2vGvPQNnyybJmiqpr1HiKX,Sorriso ao Sono,Phill Veras,0.627,0.342,2,-12.833,1,0.7000,0.16600,0.0820,0.513,80.030,248480,0.99
0,1PuLHwFZoh5qYK89I5YBdZ,I Will Wait,Mumford & Sons,0.486,0.769,1,-5.140,1,0.0119,0.00832,0.1970,0.393,131.286,276720,0.39


In [30]:
print("How similar is it to my taste?", round(felipe_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 83.8 %


In [31]:
felipe_df[felipe_df['prediction'] >0.98]

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,prediction
0,2vGvPQNnyybJmiqpr1HiKX,Sorriso ao Sono,Phill Veras,0.627,0.3420,2,-12.833,1,0.700,0.166000,0.0820,0.5130,80.030,248480,0.99
0,3Ed55g8hCz1GmqRTelIPIW,People Change,Mipso,0.374,0.2760,7,-14.945,1,0.884,0.012300,0.1220,0.4120,80.169,212853,1.00
0,4f1n0muKxA870NYsPmraLN,Don't Watch Me Dancing,Little Joy,0.503,0.2840,6,-12.920,1,0.872,0.558000,0.1430,0.5260,142.913,214413,0.99
0,0ytcZAli1HSG5qHNp76y6Y,Evaporar,Little Joy,0.623,0.2080,9,-14.083,1,0.904,0.918000,0.1200,0.2350,84.584,230800,1.00
0,78dUBuP4dmw7krQz2yEI97,Unattainable,Little Joy,0.571,0.1100,11,-12.539,1,0.964,0.001160,0.1620,0.4920,109.842,119587,1.00
0,4Qq8rERaPWkDuZsvSWR9LL,Volta,O Terno,0.387,0.1540,10,-11.680,1,0.967,0.000113,0.1130,0.3060,83.347,222107,0.99
0,2LWW6D1zabtolij6KFXQYm,Cartão Postal,Apanhador Só,0.527,0.2960,8,-10.002,0,0.918,0.000398,0.2630,0.2160,112.995,256173,1.00
0,0QUiguzo0mWdlBk6Nvujic,You Already Know,Kathryn Williams,0.662,0.2070,2,-16.176,1,0.916,0.069800,0.1010,0.4960,95.466,262533,0.99
0,4CwpP3vWh0f5VsEmvdOWib,Runaway,The National,0.583,0.4410,4,-10.321,1,0.869,0.557000,0.1170,0.3490,135.037,333787,0.99
0,6a20J6YFMYUjUZ7WnuJYZu,Cinder and Smoke,Iron & Wine,0.731,0.3000,11,-14.641,0,0.864,0.809000,0.1040,0.3980,112.178,344320,1.00


### Clara: on repeat

https://open.spotify.com/playlist/37i9dQZF1EpzPxJ69EcfPu?si=PR0kpS7VTrysJNGElmwH5g

In [32]:
#I'm going to test with Clara P's on repeat

playlist_index = 2

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_uri
#spotify:playlist:37i9dQZF1EpzPxJ69EcfPu

'spotify:user:Test_2:playlist:37i9dQZF1EpzPxJ69EcfPu'

In [33]:
clara_df = master_function(playlist_uri)
clara_df.shape

(30, 15)

In [34]:
pred = clf.predict_proba(clara_df[features])[:,1]

clara_df['prediction'] = pred
clara_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,3dUgqHR8CIRG7Ts1cyqYD0,The Wolves (Act I and II),Bon Iver,0.465,0.164,5,-16.018,1,0.7880,0.000031,0.0816,0.1230,117.235,322053,3,1.00
0,2uEgyXzyJBr5Iuvqc83d4N,And You Don’t Even Know You Hurt Me,Nick Murphy,0.178,0.029,5,-25.484,1,0.9940,0.931000,0.1010,0.0711,71.676,808503,4,0.94
0,6Kkt27YmFyIFrcX3QXFi2o,Liability,Lorde,0.587,0.229,10,-11.254,0,0.9200,0.000000,0.1000,0.3790,75.670,171728,4,0.85
0,0KpA8R3NM3N0JB4NAGbAxO,Life On Mars?,Seu Jorge,0.655,0.146,10,-16.690,1,0.8990,0.001100,0.1190,0.2790,124.104,209067,4,0.99
0,1bSpwPhAxZwlR2enJJsv7U,No Surprises,Radiohead,0.255,0.393,5,-10.654,1,0.0577,0.003610,0.1130,0.1180,76.426,229120,4,0.59


In [35]:
print("How similar is it to my taste?", round(clara_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 92.4 %


### Soundtrack for Everyday Stuff

https://open.spotify.com/playlist/2Krvp2ENVs0eAPM6ypduW7?si=jxTn8fJcSQS7-d2SFCcHow

In [36]:
playlist_index = 3

playlists = json.load(open('playlists_test.json'))
playlist_uri = playlists[playlist_index]['uri']
test = playlists[playlist_index]['like']

client_credentials_manager = SpotifyClientCredentials(client_id=client_id,client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [37]:
jo_df = master_function(playlist_uri)
jo_df.shape

(72, 15)

In [38]:
pred = clf.predict_proba(jo_df[features])[:,1]

jo_df['prediction'] = pred
jo_df.head()

,id,title,main_artist,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,prediction
0,4MoDxgAbITVgPz0vYM8LXY,Ocean,Fruit Bats,0.641,0.275,0,-13.191,1,0.8510,0.148000,0.112,0.294,82.769,198307,4,0.99
0,479LUXp1pWPOaKXjbVBtzn,Your Dead Grandfather,Fruit Bats,0.668,0.681,0,-9.447,1,0.0867,0.206000,0.101,0.571,100.006,170093,4,0.80
0,3DLbFY6UkPBAYTpV6IcAxl,Mandy from Mohawk (Wherever You May Be),Fruit Bats,0.671,0.676,7,-8.940,1,0.0809,0.007000,0.152,0.673,120.029,227427,4,0.70
0,4B5jxg6EPyZVjfZQJ9NptN,Cazadera,Fruit Bats,0.682,0.319,0,-11.664,0,0.5540,0.068700,0.083,0.369,84.023,205493,4,0.90
0,5WB0mgQhlXV3wrb4RXwYy0,Maria,Time for T,0.653,0.573,8,-7.294,1,0.6190,0.000014,0.253,0.512,128.655,263693,4,0.53


In [39]:
print("How similar is it to my taste?", round(jo_df['prediction'].mean()*100,3), "%")

How similar is it to my taste? 87.028 %
